# Assignment 3.4 ER Data science assignment

## Description of the dataset
Following dataset was found on github that was aimed to give personalized medical recommendation using a recommendation system.

[Source of medical recommedner file](https://github.com/611noorsaeed/Medicine-Recommendation-System-Personalized-Medical-Recommendation-System-with-Machine-Learning)

[Source of sentence transformer file](https://www.kaggle.com/code/aruneembhowmick/hugging-transformers-for-movie-recommendation)

## Introduction to datset and project goal
Dataset contains symptoms for medical conditions/diseases that has already been prepared for labled machine learning process.
Goal is to create a medical recommender system which based on the symptoms would recommend the user what the potential condition may be and the best course of action and perhaps even if it is urgent and requires an ambulance to pick up the user.

## Challenges
Although the dataset was highly detailed and I made use of it, I couldn't identify how exactly the recommender system was working in the github example.
Which is why I switched mid way to make use of the sentence transformer with medical bert to use a cosine similarity to make the sentence based recommender system that looked at the mentioned symptoms.

I also ran into multiple issues such as pruning with TF-IDF and numpy issues with lower casing tokens.
There's a lot of material out on the web but some of the process is outdated or was too incomprehensible for me to replicate.

At the moment I managed to make the sentence transformer work using a public medical records bert to extract and combine with cosine similarities to form the recommender system.

Next step would be to measure the accuracy of the system which we'll go more in depth in Block D as per my understanding.

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

In [6]:
df = df = pd.read_csv('Training.csv')

In [7]:
heart = df.loc[df["prognosis"] == "Heart attack"]
heart

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
290,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Heart attack
291,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Heart attack
292,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Heart attack
293,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Heart attack
294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Heart attack
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4744,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Heart attack
4785,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Heart attack
4826,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Heart attack
4867,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Heart attack


In [8]:
#Finding symptoms of heart attack
#Source of code: https://stackoverflow.com/questions/61960929/extract-only-columns-that-contains-true-value-from-a-dataframe-in-pandas
heart.loc[:,heart.any()]

,vomiting,breathlessness,sweating,chest_pain,prognosis
290,1,1,1,1,Heart attack
291,1,1,1,1,Heart attack
292,0,1,1,1,Heart attack
293,1,0,1,1,Heart attack
294,1,1,0,1,Heart attack
...,...,...,...,...,...
4744,1,1,1,1,Heart attack
4785,1,1,1,1,Heart attack
4826,1,1,1,1,Heart attack
4867,1,1,1,1,Heart attack


## Using lemmatizer to detect symptoms

In [9]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.util import bigrams


## Using clinical bert to get pain score

In [10]:
from transformers import pipeline

#src = https://huggingface.co/sid321axn/Bio_ClinicalBERT-finetuned-medicalcondition

pipe = pipeline("text-classification", model="sid321axn/Bio_ClinicalBERT-finetuned-medicalcondition")
pipe("I have chest pain and feel like vomiting, I'm in great pain")

Device set to use cpu


[{'label': 'Pain', 'score': 0.6566210985183716}]

In [11]:
chat = "I'm feeling breathless, have pain in chest and feel like vomiting"

# Convert text to lowercase
cleaned_text = chat.lower()

# Remove punctuation
cleaned_text = cleaned_text.translate(str.maketrans('', '', string.punctuation))

# Remove numbers
cleaned_text = re.sub(r'\d+', '', cleaned_text)

# Tokenize text
tokens = cleaned_text.split()

# Remove stopwords
stop_words = set(stopwords.words('english'))
filtered_tokens = [word for word in tokens if word not in stop_words]

# Lemmatize tokens
lemmatizer = WordNetLemmatizer()
lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

# Final cleaned text
print(lemmatized_tokens)

['im', 'feeling', 'breathless', 'pain', 'chest', 'feel', 'like', 'vomiting']


In [12]:
import numpy as np
# if urgency value is higher than or equal to 2 then connect with doctor
# code from https://stackoverflow.com/questions/65947702/calculate-number-of-items-in-one-list-are-in-another

urgent_symp = ["chest pain", "vomiting", "breathlessness", "sweating"]

a_list=np.array(lemmatized_tokens)
b_list=np.array(urgent_symp)

c = np.intersect1d(a_list, b_list)
c_val = c.size
print(a_list)
print(b_list)
print(c.size)

while True:
    if "breathlessness" in a_list:
        print("Let me send you an ambulance, I'm suspecting it could be a potential heart attack")
        break
    elif "chest" and "pain" in a_list:
        print("That could potentially be a heart attack")
        break
    elif c_val >= 2:
        print("Let me connect you with a doctor to get a closer look at your condition")
        break
    else:
        print("Here is what you can do to try improve your condition")
        break


['im' 'feeling' 'breathless' 'pain' 'chest' 'feel' 'like' 'vomiting']
['chest pain' 'vomiting' 'breathlessness' 'sweating']
1
That could potentially be a heart attack


## Create TF-IDF Vectors
Using TF-IDF along with KMeans clustering to create a recommender system

src: https://www.kaggle.com/code/ahmadibraheeem/movie-recommendation-system-using-kmeans-and-hc

In [13]:
symptom = pd.read_csv('symtoms_df.csv')
symptom

,Unnamed: 0,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4
0,0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches
1,1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN
2,2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN
3,3,Fungal infection,itching,skin_rash,dischromic _patches,NaN
4,4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN
...,...,...,...,...,...,...
4915,4915,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,spinning_movements
4916,4916,Acne,skin_rash,pus_filled_pimples,blackheads,scurring
4917,4917,Urinary tract infection,burning_micturition,bladder_discomfort,foul_smell_of urine,continuous_feel_of_urine
4918,4918,Psoriasis,skin_rash,joint_pain,skin_peeling,silver_like_dusting


In [14]:
#merge symptoms into a new column for tf-idf vectors
symptom['combined'] = [x for x in symptom[['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4']].to_numpy()]
symptom['combined']

0       [itching,  skin_rash,  nodal_skin_eruptions,  ...
1       [ skin_rash,  nodal_skin_eruptions,  dischromi...
2       [itching,  nodal_skin_eruptions,  dischromic _...
3        [itching,  skin_rash,  dischromic _patches, nan]
4       [itching,  skin_rash,  nodal_skin_eruptions, nan]
                              ...                        
4915    [ vomiting,  headache,  nausea,  spinning_move...
4916    [ skin_rash,  pus_filled_pimples,  blackheads,...
4917    [ burning_micturition,  bladder_discomfort,  f...
4918    [ skin_rash,  joint_pain,  skin_peeling,  silv...
4919    [ skin_rash,  high_fever,  blister,  red_sore_...
Name: combined, Length: 4920, dtype: object

In [15]:
import nltk
import re
from itertools import combinations
from nltk.tokenize import PunktTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

In [16]:
symptom2 = symptom.replace(r"_", "", regex=True)
symptom2

,Unnamed: 0,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,combined
0,0,Fungal infection,itching,skinrash,nodalskineruptions,dischromic patches,"[itching, skin_rash, nodal_skin_eruptions, ..."
1,1,Fungal infection,skinrash,nodalskineruptions,dischromic patches,NaN,"[ skin_rash, nodal_skin_eruptions, dischromi..."
2,2,Fungal infection,itching,nodalskineruptions,dischromic patches,NaN,"[itching, nodal_skin_eruptions, dischromic _..."
3,3,Fungal infection,itching,skinrash,dischromic patches,NaN,"[itching, skin_rash, dischromic _patches, nan]"
4,4,Fungal infection,itching,skinrash,nodalskineruptions,NaN,"[itching, skin_rash, nodal_skin_eruptions, nan]"
...,...,...,...,...,...,...,...
4915,4915,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,spinningmovements,"[ vomiting, headache, nausea, spinning_move..."
4916,4916,Acne,skinrash,pusfilledpimples,blackheads,scurring,"[ skin_rash, pus_filled_pimples, blackheads,..."
4917,4917,Urinary tract infection,burningmicturition,bladderdiscomfort,foulsmellof urine,continuousfeelofurine,"[ burning_micturition, bladder_discomfort, f..."
4918,4918,Psoriasis,skinrash,jointpain,skinpeeling,silverlikedusting,"[ skin_rash, joint_pain, skin_peeling, silv..."


In [17]:
symptom2['combined'] = symptom2['combined'].str.replace('_', ' ')

In [18]:
symptom2['combined'] = symptom2['combined'].astype(str).str.lower()
symptom2['combined'] = symptom2['combined'].astype(str).replace(r'[^a-zA-z0-9\s]', '', regex=True)

In [19]:
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer

#Kept getting error float object has no attribute lower
#src of solution https://stackoverflow.com/questions/74674913/how-to-solve-attributeerror-float-object-has-no-attribute-lower
symptom2['combined'] = symptom2['combined'].astype(str).str.lower()
symptom2['combined'] = symptom2['combined'].astype(str).replace(r'[^a-zA-z0-9\s]', '', regex=True)

# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=3, max_df=0.4)

tfidf_matrix = vectorizer.fit_transform(symptom['unique'])

# Normalizing the TF-IDF matrix
tfidf_matrix_norm = normalize(tfidf_matrix)
terms = vectorizer.get_feature_names_out()

tfidf_df = pd.DataFrame(tfidf_matrix_norm.toarray(), columns=terms)

tfidf_df

KeyError: 'unique'

There seems to be issues with pruning which I wasn't able to solve unfortunately
After pruning, no terms remain. Try a lower min_df or a higher max_df.
If it had functioned I would have made use of the k means clustering and used cosine similairity to find the similarity distance between user symptoms with the tf idf vectors.

## Reflection on first version
I tried to make use of the lemmatizer in the first recommender system to extract the words into a list format and cross check via creating a list of symptoms for an heart attack. There probably is a way to make it simpler and cross reference check with symptoms of different disesases without manually doing it but I didn't have enough time to explore that option.

This obviously can work but is more time consuming to do manually but gave me a good idea on based on which symptoms were identified I can recommend different course of actions. Also not quite a recommender system but more of a script.

Secondly, I tried to create a tf idf vector to run through the kmeans clustering for cosine similarity as seen here:
https://www.kaggle.com/code/ahmadibraheeem/movie-recommendation-system-using-kmeans-and-hc
but I ran into pruning issues that I couldn't resolve even after spending 2+ hours on it

## Second Exploration using huggingface sentence transformers

## Exploring Huggingface sentence transformers for condition recommendation
src: https://www.kaggle.com/code/aruneembhowmick/hugging-transformers-for-movie-recommendation

In [ ]:
#pip install sentence_transformers

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [20]:
from sentence_transformers import SentenceTransformer
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
#mean pooling to get an average of vector embeddings
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

#Embed information to recommend users similar words/results/information
def get_embeddings(sentences):
    encoded_input = tokenizer(sentences, padding = True, truncation = True, return_tensors = 'pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    sentence_embeddings = F.normalize(sentence_embeddings, p = 2, dim = 1)
    return sentence_embeddings

NameError: name 'AutoTokenizer' is not defined

In [ ]:
#Using huggingface src: https://huggingface.co/pritamdeka/S-PubMedBert-MS-MARCO
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

tokenizer = AutoTokenizer.from_pretrained('pritamdeka/S-PubMedBert-MS-MARCO')
model = AutoModel.from_pretrained('pritamdeka/S-PubMedBert-MS-MARCO')

def get_embeddings(sentences):
    encoded_input = tokenizer(sentences, padding = True, truncation = True, return_tensors = 'pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    sentence_embeddings = F.normalize(sentence_embeddings, p = 2, dim = 1)
    return sentence_embeddings

NameError: name 'AutoTokenizer' is not defined

In [ ]:
symptoms = pd.read_csv('symtoms_df.csv')
symptoms

,Unnamed: 0,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4
0,0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches
1,1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN
2,2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN
3,3,Fungal infection,itching,skin_rash,dischromic _patches,NaN
4,4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN
...,...,...,...,...,...,...
4915,4915,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,spinning_movements
4916,4916,Acne,skin_rash,pus_filled_pimples,blackheads,scurring
4917,4917,Urinary tract infection,burning_micturition,bladder_discomfort,foul_smell_of urine,continuous_feel_of_urine
4918,4918,Psoriasis,skin_rash,joint_pain,skin_peeling,silver_like_dusting


In [ ]:
sym = symptoms.dropna()

In [ ]:
#to list src: https://stackoverflow.com/questions/42316088/bokeh-attributeerror-dataframe-object-has-no-attribute-tolist
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
sym_embeddings = model.encode(sym[['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4']].values.tolist())

In [ ]:
#Use the medical Bert as the model for the sentence transformer
#to list src: https://stackoverflow.com/questions/42316088/bokeh-attributeerror-dataframe-object-has-no-attribute-tolist
model = SentenceTransformer('pritamdeka/S-PubMedBert-MS-MARCO')
sym_embeddings = model.encode(sym[['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4']].values.tolist())

In [ ]:
#using cosine similarity to get recommendations
def get_recommendations(query, embeddings, df, top_n = 5):
    query_embedding = model.encode([query])
    similarities = cosine_similarity(query_embedding, embeddings)
    top_indices = similarities[0].argsort()[-top_n:][::-1]
    return df.iloc[top_indices]

In [ ]:
#Query of what the potential condition could be based on the user message to the chatbot or the query
query = "I have phlegm and chills"
recommendations = get_recommendations(query, sym_embeddings, sym)
recommendations['Disease']

974     Malaria
154     Malaria
1384    Malaria
1794    Malaria
2204    Malaria
Name: Disease, dtype: object

In [ ]:
#Extract only the first row and mention the potential condition
rec = recommendations['Disease'].iloc[0]
rec
print(f"Judging from your symptoms, the system believes that it could potentially be {rec}")

Judging from your symptoms, the system believes that it could be Malaria


In [ ]:
#Severity score that I generated
#in real world scenario they would be created by a medical professional who judges the severity ranking per disease
severity = pd.read_csv('severity.csv')
severity

,Disease,Rank
0,Paralysis (brain hemorrhage),6
1,Tuberculosis,5
2,Common Cold,3
3,Pneumonia,5
4,Heart attack,7
5,Urinary tract infection,3
6,AIDS,4
7,Allergy,3
8,Acne,1
9,Fungal infection,2


In [ ]:
# Find the matching severity rank in severity dataframe with sym dataframe
# Add a new column in the updated dataframe that has the rank according to it
# Used the help of gemini to create the function
def add_rank_to_symptoms_df(sym, severity_df):
  """Adds a rank column to symptoms_df based on matching Disease names in severity_df.

  Args:
    symptoms_df: The symptoms DataFrame.
    severity_df: The severity DataFrame containing Disease names and ranks.

  Returns:
    The symptoms DataFrame with an added 'Rank' column.
  """

  # Merge the two DataFrames on the 'Disease' column
  merged_df = pd.merge(sym, severity_df[['Disease', 'Rank']], on='Disease', how='left')

  # Rename the 'Rank' column to avoid conflicts if it already exists
  merged_df = merged_df.rename(columns={'Rank': 'Rank_Severity'})

  return merged_df


updated_df = add_rank_to_symptoms_df(sym, severity)

In [ ]:
#Calling the severity rank depending on the disease/condition
rec_val = severity[severity["Disease"] == rec]["Rank"].values[0]
rec_val
print(f"Severity Level 1: Non Serious, Severity Level 4: Medium, Severity Level 7: Very High")
print(f"The suggested severity level is {rec_val}")

Severity Level 1: Non Serious, Severity Level 4: Medium, Severity Level 7: Very High
The suggested severity level is 3


In [ ]:
#Recommended course of action

while True:
    if rec_val > 6:
        print("That sounds concerning let me send you an ambulance")
        break
    elif rec_val > 4:
        print("Let me connect you with a doctor for a more detailed diagnosis")
        break
    elif rec_val >1:
        print("Can you tell me how long it's been going on?")
        break
    elif rec_val <= 1:
        print("Here is what you can do:")
        break

Can you tell me how long it's been going on?
